<a href="https://colab.research.google.com/github/kd303/trnsfrmr_pytrch_end_p1/blob/main/session_06_attentions/session_06_atten_model_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/data')

Mounted at /content/data


In [4]:
cd /content/data/MyDrive/

/content/data/MyDrive


In [15]:
!pwd

/content


In [7]:
cd sample_data/

/content/sample_data


In [12]:
!mkdir ses_6_data

In [13]:
cd ses_6_data

/content/data/MyDrive/ses_6_data


In [14]:
!wget http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz

--2021-11-27 17:48:55--  http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8254496 (7.9M) [application/x-gzip]
Saving to: ‘Question_Answer_Dataset_v1.2.tar.gz’

Question_Answer_Dat 100%[===================>]   7.87M  2.02MB/s    in 4.9s    

2021-11-27 17:49:00 (1.61 MB/s) - ‘Question_Answer_Dataset_v1.2.tar.gz’ saved [8254496/8254496]



In [15]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2021-11-27 17:49:11--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M  48.3MB/s    in 1.1s    

2021-11-27 17:49:13 (48.3 MB/s) - ‘quora_duplicate_questions.tsv’ saved [58176133/58176133]



In [ ]:
!tar -xvf Question_Answer_Dataset_v1.2.tar.gz

In [ ]:
## !find /content/drive quora_duplicate_questions.tsv

In [1]:
## 

quora_data_file = '/content/drive/MyDrive/ses_6_data/quora_duplicate_questions.tsv'
cmu_qna_file = '/content/drive/MyDrive/ses_6_data/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt'



## Bit of my frustration and something you can ignore..

Since My life has been spent in creating a word whcih is made of object, I cannot resist but creating classes and putting everything under a neat and tidy classes. Although I feel python is faking OOPs (that damn reference to self explicitly delcared in every memthod, compared to Java (old flame), I have accepted my fate :)

#### Essentially two classes#
1. ```language_dictonary``` - Sweetly ignoring DataLaoder and build_vocab for a custom built cluegy single threaded model.

2. ```dataset_parser``` - Just a helper class loading TSV and dealing with dirty business.

In [2]:
import re
import unicodedata

class dataset_parser:
    def __init__(self, file_path, MAX_LEN):
        self.file_path = file_path
        self.MAX_LEN = MAX_LEN

    def normalizeString(self, s):
        s = self.unicodeToAscii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        return s


    def is_max_length_match(self, p):
        return (len(p[0].split(' ')) < self.MAX_LEN
                and len(p[1].split(' ')) < self.MAX_LEN)

    ## I found some characters which I dont understand, thou shall ignore those.
    def unicodeToAscii(self, s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

    def filter_pairs(self, pairs):
        return [p for p in pairs if self.is_max_length_match(p)]

    def parseFile(self, file_type):
        lines = open(self.file_path, encoding='utf-8').read().strip().split('\n')

        # for Quora dataset, ignore rest of the columns
        if file_type == 'QUORA':
            pairs = [[self.normalizeString(l.split('\t')[3]), self.normalizeString(l.split('\t')[4])] for l in lines[1:] if len(l.split('\t')) > 4]
        else:  # CMU
            pairs = [[self.normalizeString(l.split('\t')[1]), self.normalizeString(l.split('\t')[2])] for l in lines[1:] if len(l.split('\t')) > 2]
            # [pair for pair in pairs if len(pair[0].split(' ')) < MAX_LEN and len(pair[1].split(' ')) < MAX_LEN]
        return self.filter_pairs(pairs)

class language_dictionary:
    def __init__(self, dataset_name, MAX_LEN):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_wrds = 2
        self.MAX_LEN = MAX_LEN
        self.dataset_name_ = dataset_name

    @property
    def dataset_name(self):
        return self.dataset_name_

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_wrds
            self.word2count[word] = 1
            self.index2word[self.n_wrds] = word
            self.n_wrds += 1
        else:
            self.word2count[word] += 1

In [3]:
MAX_LENGTH = 20
quora = dataset_parser(quora_data_file, MAX_LENGTH)
pairs = quora.parseFile('QUORA')

input_sent = language_dictionary('QUORA_INPUT', MAX_LENGTH)
output_sent = language_dictionary('QOURA_OUTPUT', MAX_LENGTH)

for pair in pairs:
    input_sent.addSentence(pair[0])
    output_sent.addSentence(pair[1])


In [4]:
sample_input_sent = pairs[8379][0]
sample_output_sent = pairs[8379][1]

In [5]:
sample_input_sent, sample_output_sent

('where can adderalls be found in india without prescription and without adhd ?',
 'can you get adderal without a prescription in india ?')

In [6]:
input_indices = [input_sent.word2index[word] for word in sample_input_sent.split(' ')]
output_indices = [output_sent.word2index[word]  for word in sample_output_sent.split(' ')]

In [7]:
input_indices,  output_indices

([304, 23, 9352, 55, 4345, 10, 13, 289, 3765, 48, 289, 8064, 14],
 [27, 60, 305, 8955, 476, 56, 8956, 10, 435, 13])

In [8]:
SOS_token = 0
EOS_token = 1

input_indices.append(EOS_token)
output_indices.append(EOS_token)

In [9]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
device

device(type='cpu')

In [11]:
input_tensor = torch.tensor(input_indices, dtype=torch.long, device=device)
ouput_tensor = torch.tensor(output_indices, dtype=torch.long, device=device)

## Variable understanding

1. input_size = size of the vocabulary
2. hidden_size = hiddent layer for embedding size
3. hidden_ouput = out put dimension from LSTM/GRUs.

In [12]:
input_size = input_sent.n_wrds
hidden_size = 256
hidden_output = 128
input_size

53793

## We will be faking a batch, here the embeddeing vector will be (vocab_size, hidden_size), what we really need is a batch (Vocab_size, batch_size, hidden_size)

* Is it really needed if it is a non-GPU operation or say we are running on CPU (To be tested on CPU iteration)??*

* For NLP the pytorch deals with batch size as second parameter, whilst for CNNs/Images usually batch size is first parameter

As per the [documentation on RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN), if ```batch_first = True``` then the output will be provided as ```[batch_size, vocab, hidden_size]``` However this does not apply 
    1. to Embedding.
    2. nn.Linear layer expects batch size as first positional argument

```
    torch.nn.RNN(*args, **kwargs)
```

Note from the documentation:

Keep in mind that only a limited number of optimizers support **sparse gradients**: currently it’s optim.SGD (CUDA and CPU), optim.SparseAdam (CUDA and CPU) and optim.Adagrad (CPU)
  - Sparse Gradients - implies network not receiving strong signals in backpropagation to tune the network, basically indicates vanishing gradient problem

In [13]:
embedding = nn.Embedding(input_size, hidden_size).to(device)
gru = nn.GRU(hidden_size, hidden_output).to(device)

In [21]:
print(embedding)
print(gru)
print(input_tensor.shape)


Embedding(53793, 256)
GRU(256, 128)
torch.Size([14])


In [15]:
## useless as does not have batch information, it will throw an error when passed to RNN/GRU cell 
# embedding_input = embedding(input_tensor) 
# print(embedding_input.shape)

torch.Size([14, 256])


# Faking Batch using View/Squeeze

### Difference between View and Squeeze
1. when Dimensionality unknown use Unsequeze - takes a single dimension - so to add one dimension at a given position
2. View needs is a view created on original tensor with almost equivalen of reshape operation. View avoids data copying of underlying tensor

All [View ops](https://pytorch.org/docs/stable/tensor_view.html) in PyTorch are follows of not copying the data, so reshape, unsqueeze, view all are view ops. Some view operations do have special cases where they may return non-contigious tensor which upon calling ```.contigious()``` would return a contigious

[Contingious Tensor](https://discuss.pytorch.org/t/contigious-vs-non-contigious-tensor/30107) has certain behaviour on copying the data and performance. How a tensor is laid out in memory and reading performance.

In [16]:
embedding_input = embedding(input_tensor.view(-1, 1))
embedding_input.shape

torch.Size([14, 1, 256])

## Understanding the inputs:

1. embedding input - of dimensions (embedding_size, batch, hidden_size) as declared in embedding class initialization.
2. Hidden vector - basically previous output
uni directional LSTM **if** it is 2 layers and bi-directional then the vector should be of dimensions ```(4,1, hidden_output)``` here it will be ```(1, 1, hidden_output)```

  ```(1*num_layers, batch_size, hidden_ouput)```        or 
  ```(2*num_layers, batch_size, hidden_ouput)```


In [17]:
prv_out_put = torch.zeros(1,1,hidden_output, device=device)
output, hidden = gru(embedding_input, prv_out_put)

In [18]:
prv_out_put.shape, output.shape, hidden.shape

(torch.Size([1, 1, 128]), torch.Size([14, 1, 128]), torch.Size([1, 1, 128]))

## A word about storing Encoder ouputs

1. we are passing the tokens ```MAX_LENGTH``` and each output to GRU layer needs to be saved in order to calculate Attention Matrix. - keep clam and storing on :)

**** Dimensions *****

single output is (1, batch_size, hidden_ouput)
So the tensor requires for storing is (MAX_LENGTH, hidden_output)

In [19]:
input_tensor.shape, output.shape, hidden.shape, input_tensor.size()

(torch.Size([14]),
 torch.Size([14, 1, 128]),
 torch.Size([1, 1, 128]),
 torch.Size([14]))

In [25]:
encoder_outputs = torch.zeros(MAX_LENGTH, hidden_output, device=device)

## for each word in input tensor   GRU(256, 128)
for i in range(input_tensor.size()[0]):
  embedding_in = embedding(input_tensor[i])
  print("Embedding", embedding_in.shape)
  output, hidden_ou = gru(embedding_in, hidden)
  print("Output : ", output.shape, "Hidden : ", hidden_ou.shape )

Embedding torch.Size([256])


IndexError: ignored

### Above Error are due to embedding size is [256], what is expected is ```[1, 1, 256]``` where ```1``` - is for ```word```, ```1``` - ```batch``` and ```256``` is ```embedding size```

1. convert the embedding to 1, 256
2. When not using ```nn.Module```  the loop is taken care ..as told by *the Oracle* -> Need to find out **why**
3. Why does all encoder_outputs need ```+``` ?

In [24]:
encoder_outputs = torch.zeros(MAX_LENGTH, hidden_output, device=device)
hidden_in = torch.zeros(1, 1, hidden_output, device = device)
## for each word in input tensor   GRU(256, 128)
print(" loop for : ",input_tensor.size()[0])
for i in range(input_tensor.size()[0]):
  # print("input_tensor shape : ", input_tensor[i])
  embedding_in = embedding(input_tensor[i].view(-1, 1))
  # print("Embedding", embedding_in.shape, " hidden output: ", hidden_in.shape)
  output, hidden_ou = gru(embedding_in, hidden_in)
  # print("Output : ", output.shape, " Hidden gru : ", hidden_ou.shape, "output [0, 0] : ", output[0, 0].shape)
  encoder_outputs[i] += output[0, 0]

print("encoder_outputs : ", encoder_outputs.shape)

 loop for :  14
encoder_outputs :  torch.Size([20, 128])


# Decoder with Attention








In [26]:
decoder_input = torch.tensor([[SOS_token]] , device = device)
decoder_hidden = hidden_in
decoder_input.shape, decoder_hidden.shape

(torch.Size([1, 1]), torch.Size([1, 1, 128]))

In [27]:
output_size = output_sent.n_wrds
embedding_out = nn.Embedding(output_size, hidden_output).to(device)
embedded_out = embedding_out(decoder_input)
embedded_out.shape

torch.Size([1, 1, 128])

## Attention Layer  Def

1. concatinating output of encoder and output of decoder will be passed through fully connected layer
So the **Dimensions would be** 
  - 2*hidden_ouput
  - n_wrds for input sentence, which is 14 or could be upto max_length
2. Then Matrix multiplication in batch 

In [30]:
attn_layer = nn.Linear(2*hidden_output, MAX_LENGTH).to(device)
attn_lyr_wgts = attn_layer(torch.cat((embedded_out[0], decoder_hidden[0]), 1))

In [31]:
attn_lyr_wgts.shape

torch.Size([1, 20])

In [32]:
hidden_output, embedded_out[0].shape, decoder_hidden[0].shape

(128, torch.Size([1, 128]), torch.Size([1, 128]))

In [33]:
(torch.cat((embedded_out[0], decoder_hidden[0]), 1)).shape

torch.Size([1, 256])

In [34]:
import torch.nn.functional as F

attn_layer = nn.Linear(2*hidden_output, MAX_LENGTH).to(device)
attn_lyr_wgts = attn_layer(torch.cat((embedded_out[0], decoder_hidden[0]), 1))
attn_lyr_sftmx = F.softmax(attn_lyr_wgts, dim=1)
print(attn_lyr_sftmx.shape)
attn_lyr_sftmx

torch.Size([1, 20])


tensor([[0.0488, 0.0541, 0.0854, 0.0514, 0.0484, 0.0267, 0.0236, 0.0566, 0.0498,
         0.0355, 0.0231, 0.0589, 0.0475, 0.0688, 0.0403, 0.0749, 0.0581, 0.0563,
         0.0467, 0.0450]], grad_fn=<SoftmaxBackward0>)

In [35]:
 attn_lyr_wgts.shape, attn_lyr_wgts.shape

(torch.Size([1, 20]), torch.Size([1, 20]))

In [38]:
### softmax probability should 0?

sum([0.0488, 0.0541, 0.0854, 0.0514, 0.0484, 0.0267, 0.0236, 0.0566, 0.0498, 0.0355, 0.0231, 0.0589, 0.0475, 0.0688, 0.0403, 0.0749, 0.0581, 0.0563, 0.0467, 0.0450])

0.9998999999999999

In [39]:
import torch.nn.functional as F

attn_layer = nn.Linear(2*hidden_output, MAX_LENGTH).to(device)
attn_lyr_wgts = attn_layer(torch.cat((embedded_out[0], decoder_hidden[0]), 1))
attn_lyr_sftmx = F.softmax(attn_lyr_wgts, dim=1)
print(attn_lyr_sftmx.shape)
attn_lyr_sftmx

torch.Size([1, 20])


tensor([[0.0518, 0.0199, 0.0714, 0.0186, 0.0421, 0.0380, 0.0474, 0.0383, 0.0322,
         0.0711, 0.0596, 0.0356, 0.0625, 0.0548, 0.0690, 0.0629, 0.0580, 0.0971,
         0.0346, 0.0351]], grad_fn=<SoftmaxBackward0>)

In [85]:
attn_lyr_sftmx.shape, encoder_outputs.shape

(torch.Size([1, 20]), torch.Size([20, 128]))

> ### Will throw an error crying for b, B stand for batch in Bmm if I dont add the batch, batch indicates first argument here - different from RNN cells...


> ### torch.bmm(attn_lyr_sftmx, encoder_outputs) using unsqueeze and view - why cause I can! but it shows how combersome it







In [40]:

attn_lyr_sftmx.unsqueeze(0).shape,  encoder_outputs.view(-1, 20 ,128).shape,  encoder_outputs.unsqueeze(0).shape

(torch.Size([1, 1, 20]), torch.Size([1, 20, 128]), torch.Size([1, 20, 128]))

In [41]:
bmm = torch.bmm(attn_lyr_sftmx.unsqueeze(0), encoder_outputs.unsqueeze(0) )
bmm.shape

torch.Size([1, 1, 128])

### Input to GRU Layer for encoder is 
 - s1 - concating attention and zero input tensor
 - c1 - atten applied

In [42]:
embedded_out.shape, bmm.shape

(torch.Size([1, 1, 128]), torch.Size([1, 1, 128]))

In [43]:
in_gru_layer = nn.Linear(128 * 2, 256).to(device)
in_gru_lyr = in_gru_layer(torch.cat((embedded_out[0], bmm[0]), 1))
print(in_gru_lyr.shape)
in_gru_lyr = in_gru_lyr.unsqueeze(0)
in_gru_lyr.shape, decoder_hidden.shape

torch.Size([1, 256])


(torch.Size([1, 1, 256]), torch.Size([1, 1, 128]))

In [50]:
decdr_gru = nn.GRU(256, 256).to(device)
output, decoder_hidden = decdr_gru(decoder_hidden, in_gru_lyr)
print(" output : ", output.shape, "decoder_hidden : ", decoder_hidden.shape)


 output :  torch.Size([1, 1, 256]) decoder_hidden :  torch.Size([1, 1, 256])


In [51]:
output_word_layer = nn.Linear(256, output_sent.n_wrds).to(device)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim=1)
output.shape, output, output.data.topk(1)

(torch.Size([1, 49147]),
 tensor([[2.2734e-05, 2.4490e-05, 2.0388e-05,  ..., 1.8827e-05, 2.0228e-05,
          1.8521e-05]], grad_fn=<SoftmaxBackward0>),
 torch.return_types.topk(values=tensor([[3.3170e-05]]), indices=tensor([[33715]])))

In [61]:
print(type(output.data.topk(1)))

<class 'torch.return_types.topk'>


In [71]:
# Printing top 10 words with probabilty 
topv, topi = output.data.topk(10)
# print(topv.shape)
# print(topi.shape)
for idx, i in enumerate(topi[0]):
  wrd = output_sent.index2word[i.item()]
  print('word : ', wrd, ' probability : ', topv[0][idx])

word :  islanding  probability :  tensor(3.3170e-05)
word :  logx  probability :  tensor(3.2584e-05)
word :  meadowbrook  probability :  tensor(3.2089e-05)
word :  cyberpunk  probability :  tensor(3.1560e-05)
word :  audrey  probability :  tensor(3.1240e-05)
word :  enviromental  probability :  tensor(3.0541e-05)
word :  myregistry  probability :  tensor(3.0099e-05)
word :  trams  probability :  tensor(3.0059e-05)
word :  vectors  probability :  tensor(2.9853e-05)
word :  prelude  probability :  tensor(2.9403e-05)


In [57]:
topv, topi = output.data.topk(1)
print(topv.shape)
print(topi.shape)
for i in topi:
  wrd = output_sent.index2word[i.item()]
  print(wrd)

torch.Size([1, 1])
torch.Size([1, 1])
islanding
